In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#            print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
print("tensorflow version :",tf.__version__)

In [ ]:
import tensorflow_hub as hub

print("version of  tensorflow hub =",hub.__version__)

In [ ]:
labels_csv=pd.read_csv("/kaggle/input/dog-breed-identification/labels.csv")
labels_csv

In [ ]:
labels_csv["breed"].value_counts().plot.bar(figsize=(20,10))

# Getting images and their labels 

In [ ]:
labels_csv

In [ ]:
filenames=["/kaggle/input/dog-breed-identification/train/"+fname+".jpg"for fname in labels_csv["id"]]

filenames[:1]
# Image.open(filenames[1])

In [ ]:
from PIL import Image

# Image.open(filenames[1])

In [ ]:
a=len(os.listdir("/kaggle/input/dog-breed-identification/train"))
b=len(filenames)
if(a==b):
    print("continue")
else:
    print("kindly verify data integrity")
        

In [ ]:
labels=labels_csv["breed"].to_numpy()
len(labels)

In [ ]:
unique_breeds=np.unique(labels)
len(unique_breeds)

In [ ]:
# turmning a single value to an array of booleans 
print(labels[0])
labels[0]==unique_breeds

In [ ]:
# turning every label to a boolean label 

boolean_labels=[label==unique_breeds for label in labels ]
boolean_labels[:1]

In [ ]:
len(boolean_labels), len(labels)

In [ ]:
print(labels[0])

print(np.where(unique_breeds==labels[0]))

print(boolean_labels[0].argmax())

print(boolean_labels[0].astype(int))


In [ ]:
print (labels[2] )

print ( boolean_labels[2].astype(int))

In [ ]:
filenames[:1]

# Creating our own validation set and performing train test split on the given data set

* creating X and y
* mapping X to filenames
* mapping y to boolean_labels

In [ ]:
X=filenames
y=boolean_labels

In [ ]:
len(X)

In [ ]:
len(y)

* we are going to first work on ~ 1000 images and then increase as needed 

In [ ]:
NUM_IMAGES = 1000 
# @ param{type:"slider",min:1000, max:10000, step:1000}

In [ ]:
#  performing train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train , y_val = train_test_split(X[:NUM_IMAGES]
                                                  , y[:NUM_IMAGES],
                                                    test_size=0.2,
                                                      random_state=42)

len(X_train) , len(y_train) , len(X_val) , len(y_val)

In [ ]:
print(X_train[:1])
print(y_train[:1])

# Pre Processing The Images 
In order to preprocess the images we'll write a function that converts the images to tensors 
1. Take an image filepath as input
2. Using TensorFlow read that file and store in a variable `image`
3. turn our `image` (jpg) into tensor
4. convert the colour channel values from 0-255 to 0-1
5. resize the `image` to be a shape of (224 , 224)
6. return the modified `image`

In [ ]:
# define image size 

IMG_SIZE = 224

def process_image(image_path,img_size=IMG_SIZE):
    """
    takes an image path and returns a tensor for that image 
    """
    
#     reading an image 
    image=tf.io.read_file(image_path)
    
#     convert the jpeg image to numerical tensors with 3 colour channels RGB
    image=tf.image.decode_jpeg(image,channels=3)
    
#     convert the values from 0-255 to 0-1
    image=tf.image.convert_image_dtype(image,tf.float32)
    
#     resize the image to img_size
    image=tf.image.resize(image,size=[img_size,img_size])
    
    return image

## Convrting the data into mini batches of size 32

In [ ]:
def get_image_label(image_path,label):
    """
    takes an image processes it and returns the (image , label)
    """
    image=process_image(image_path)
    return image,label
    

In [ ]:
get_image_label(X[42],y[42])

## Creating mini batches for training the models

In [ ]:
# declaring batch size 32

BATCH_SIZE=32

# creating a fn to convert dat into bacthes 

def create_data_batches(X,y=None,batch_size=BATCH_SIZE,valid_data=False,test_data=False):
    """
    creates batches of size 32 out of images X and labels y
    shuffles the data if its training set but not if it is a validation set 
    also accepts test_data as input i.e data with no labels 
    """
    if test_data==True:
        print("creating test data batches ....")
        data=tf.data.Dataset.from_tensor_slices((tf.constant(X)))
#         the above line converts file path tensors to tensor data frame 
        data_batch=data.map(process_image).batch(batch_size)
        return data_batch    
#     if the valid data set is given ,no need to shuffle it 
    elif valid_data==True :
        print("ceretaing validation data set ......")
        data=tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y))) #label
        data_batch=data.map(get_image_label).batch(batch_size)
#         data_batch=data.map(get_image_label).batch(batch_size)
        return data_batch
    else:
        print("creating training batches ... ")
        data=tf.data.Dataset.from_tensor_slices((tf.constant(X),tf.constant(y)))
        
        data = data.shuffle(buffer_size=len(X))
        data=data.map(get_image_label)
#         data=data.map(process_image)
#         creating data_bacth 
        data_batch=data.batch(batch_size)
        return data_batch

In [ ]:
X_train[:1],y_train[:1]
train_data=create_data_batches(X_train,y_train)
# X_train.type()
# y_train.dtype()
val_data=create_data_batches(X_val,y_val,valid_data=True)

In [ ]:
train_data.element_spec, val_data.element_spec

#  Visualizing data batches 

In [ ]:
import matplotlib.pyplot as plt

# showing 25 images
def show_25_images(images,labels):
    """
    displays 25 images
    """
    plt.figure(figsize=(10,10))
    
    for i in range(25):
        ax=plt.subplot(5,5,i+1)
        plt.imshow(images[i])
        plt.title(unique_breeds[y[i].argmax()])
        plt.axis("off")

In [ ]:
train_images,train_labels = next(train_data.as_numpy_iterator())

In [ ]:
len(train_images),len(train_labels)

In [ ]:

show_25_images(train_images,train_labels)

In [ ]:
val_images,val_labels=next(val_data.as_numpy_iterator())

In [ ]:
show_25_images(val_images,val_labels)

## Building a model
b4 building few steps :
    * The input shape (our image shape , in the form of Tensors to our model )
    * The output shape (the image labels , in the form of Tensors to our model )
    * The URL of the model we want to use

In [ ]:
INPUT_SHAPE=[None,IMG_SIZE,IMG_SIZE,3]
#           batch,height,width,colour channels
OUTPUT_SHAPE=len(unique_breeds)

# setup  model url 
MODEL_URL="https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5"

#  building a function for model fitting and training and testing 
* takes input shape , o/p shape and model chosen as parameter 
* defines the layers in keras model in a sequential fashion
* compiles the model 
* builds the model 
* returns the model 

In [ ]:
# creating a fn which builds a keras model 
def create_model(input_shape=INPUT_SHAPE,output_shape=OUTPUT_SHAPE,model_url=MODEL_URL):
    print("building  model with ",model_url)
#     setting up the model layer 
    model =tf.keras.Sequential([hub.KerasLayer(model_url), #layer 1 -> input layer 
        tf.keras.layers.Dense(units=output_shape,
                             activation="softmax") # layer 2 -> output layuer 
        
    ])
#     compiling the model 
#     model.compile(
#                 loss =tf.keras.losses.CategoricalCrossentropy(),
#                  optimizers=tf.keras.optimizers.Adam(),
#                  metrics=["accuracy"]
#                  )
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     building the model 
    model.build(input_shape)
    return model

In [ ]:
model = create_model()
model.summary()


#  creating call back functions 
* call back functons are helper fns used to by the model to :
    1. save its progress 
    2. stop early
    3. check its progress

# tensor board  call back function
steps:
    1. load the tensorboard notebook extension
    2. create a tensorboard callback which is able to save logs to save logs in a directory and pass it to our model's `fit()` fn
    3. visualize our models training  logs 

In [ ]:
# load tensorboard 
%load_ext tensorboard

In [ ]:
import datetime
def create_tensorboard_callback():
    logdir=os.path.join("/kaggle/working/logs",
                       datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    return tf.keras.callbacks.TensorBoard(logdir)

# early stopping call back 
* early stopping helps stop our model from overfitting

In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                               patience = 3)

In [ ]:
 NUM_EPOCHS=100
    

# lets cerate a fn which trains a model :
* create a fn using create_model()
        
* setup a tensor board callback using `create_tensorboard_callback()`
* call the `fit()` fn on our model passing it the training ,validation             data ,no. of epochs  to  train (`NUM_EPOCHS`) and the call back we               would like to use 
* return the model 

In [ ]:
# building a fn to train a model and rreturn the trained model


def train_model():
    """
    trains a given model and returns the trained model 
    """

    
    #     create the model 
    
    model=create_model()    

    
    #     creating a new tensorboard session everyime we train a model 
    
    tensorboard=create_tensorboard_callback()

    
    #     fitting the model to the data
    
    model.fit(x=train_data,
             epochs=NUM_EPOCHS,
             validation_data=val_data,
             validation_freq=1,
             callbacks=[tensorboard,early_stopping])

    
    #     return the fitted model 
    
    return model

In [ ]:
model=train_model()

In [ ]:
logs="/kaggle/working/logs"

In [ ]:
# %load_ext tensorboard
%reload_ext tensorboard

In [ ]:
tensorboard --logdir=logs --port=6008

In [ ]:
predictions = model.predict(val_data,verbose=1)
predictions

# we'll create a function to show the real observed and prdicted value on the val_data

**Note:** prediction probablities is also known as confidence level


In [ ]:
def get_pred_label(prediction_probablities):
    
    """
    turns an array of prediction probablity into a label 
    """
    
    return unique_breeds[np.argmax(prediction_probablities)]

In [ ]:
pred_label=get_pred_label(predictions[0])
pred_label
print(f'name of dog breed  {pred_label}')

since our validation data set is still in batches we first need to unbacthify it to make predictions on the validation images and then compare these predictions to the validation labels (truth labels)

In [ ]:
def unbatchify(data):
    
    images=[]
    
    labels=[]
    
    for image,label in data.unbatch().as_numpy_iterator():
        images.append(image)
        labels.append(unique_breeds[np.argmax(label)])
    return images,labels
    

In [ ]:
val_images,val_labels=unbatchify(val_data)
val_images[0]
val_labels[0]


In [ ]:
get_pred_label(val_labels[0])

In [ ]:
def plot_pred(n=1):
    """
    view the prediction , true value ,image for a sample n
    """
    pass
#     print(f'true label :{})

In [ ]:
def plot_pred(labels ,images, n=0,prediction_probabilities=predictions):
    """
    view the prediction,truth label and image for sample n
    """
    images_=val_images[n]

    
    pred_label=get_pred_label(predictions[n])
    print(f'the predicted label is {pred_label}')
    
    true_labels=val_labels[n]
    
    print(f'the actual label is {true_labels}')
    
    print(f'the % of match is {np.max(predictions[n])*100} %')
    
    
    if true_labels==pred_label:
        colour="green"
    else:
        colour="red"
        
    

    plt.axis('off')
    
    plt.title(true_labels, fontsize=16, fontweight="bold", color=colour)
    plt.imshow(images[n])


In [ ]:
plot_pred(images=val_images,
         labels=val_labels,
          n=77)

In [ ]:
def save_model(model,suffix=None):
    """
    saves given model in the model directory
    """
    model_dir=os.path.join("/kaggle/working/model",
                          datetime.datetime.now().strftime("%d%m%Y-%H%M%s"))
    model_path=model_dir+"-"+suffix+".h5"
    
    print(f"svaing model to {model_path}...")
    
    model.save(model_path)
    
    return model_path


In [ ]:
def load_model(model_path):
    """
    loads a saved model from its path
    """
    print(f"loading saved model from {model_path} ")
    
    model=tf.keras.models.load_model(model_path,
                                    custom_objects={"KerasLayer":hub.KerasLayer})
    
    return model

In [ ]:
mp=save_model(model,suffix="1000-images-mobilenetv2")

In [ ]:
pretrained_model=load_model(mp)

In [ ]:
# evaluating the pre saved model 
model.evaluate(val_data)

In [ ]:
# evaluating the saved model 

pretrained_model.evaluate(val_data)

In [ ]:
full_data=create_data_batches(X,y)

In [ ]:
full_data

In [ ]:
full_model=create_model()

In [ ]:
full_model_tensorboard= create_tensorboard_callback()

full_model_early_stopping= tf.keras.callbacks.EarlyStopping(monitor="accuracy",
                                                            patience=3)

In [ ]:
full_model.fit(x=full_data,
              epochs=NUM_EPOCHS,
              callbacks=[full_model_tensorboard,full_model_early_stopping])

In [ ]:
# saving the fully trained model 

save_model(full_model,suffix="trained-on-full-10222-images-mobilenetv2")

In [ ]:
# saved_full_model=load_model("/kaggle/working/model/28092023-20241695932684-trained-on-full-10222-images-mobilenetv2.h5")

# making predictions on the test data set 

In [ ]:
test_path="/kaggle/input/dog-breed-identification/test/"


In [ ]:
test_filename=[test_path + fname for fname in os.listdir(test_path)]

test_filename[0]

len(test_filename)

In [ ]:
test_data=create_data_batches(test_filename,test_data=True)

In [ ]:

test_data

In [ ]:
test_predictions=full_model.predict(test_data,verbose=1)

In [ ]:
test_predictions[1]

In [ ]:
test_predictions.shape

In [ ]:
preds_df = pd.DataFrame(columns=["id"]+list(unique_breeds))
preds_df.head(1)

In [ ]:
test_ids= [os.path.splitext(path)[0] for path in os.listdir(test_path)]

In [ ]:
preds_df["id"]=test_ids

In [ ]:
preds_df.head(1)

In [ ]:
preds_df[list(unique_breeds)]=test_predictions
preds_df.head()

In [ ]:
output=preds_df.to_csv("/kaggle/working/submission1.csv",index=False)
